In [19]:
import pandas as pd
from scipy.spatial import distance

# load raw datafiles into DataFrames
roads = pd.read_csv("../data/raw/_roads3.csv")
bridges = pd.read_excel('../data/raw/BMMS_overview.xlsx')

In [69]:
df = roads.loc[(roads["road"] == "N1") | (roads["road"] == "N2")]
df.head()

,road,chainage,lrp,lat,lon,gap,type,name
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing


In [70]:
rest= roads[2225:51348]
LrpN12=df['lrp'].unique().tolist()
lrpRest=rest['lrp'].tolist()
commonLrps=[]
for lrp in LrpN12:
    for lrprest in lrpRest:
        if lrp ==lrpRest:
            commonLrps.append(road)
commonLrps            

[]

In [31]:
Roads = roads['road'].unique().tolist()

names = df['name'].tolist()

side_roads=[]
for road in Roads:
    for name in names:
        if road in name:
            side_roads.append(road) 
print("there are", len(Roads), " roads, and ", len(side_roads), "side roads to N1 and N2")

there are 847  roads, and  128 side roads to N1 and N2


In [43]:
sideroads_df = roads[roads['road'].isin(side_roads)]

sideroads_ends = sideroads_df.loc[(sideroads_df["lrp"] == "LRPE") & (sideroads_df["chainage"] > 25)]
print(len(sideroads_ends))
sideroads_tokeep = sideroads_ends['road'].tolist()

sideroads_df = sideroads_df.loc[(sideroads_df['road'].isin(sideroads_tokeep))]


29


In [45]:
sideroads_df["length"] = sideroads_df["chainage"]*1000 
sideroads_df = sideroads_df.reset_index(drop=True)


for i in range((len(sideroads_df)-1),0,-1):
    if sideroads_df["road"][i] == sideroads_df["road"][i-1]:
        sideroads_df.loc[i,"length"] = sideroads_df.loc[i,"length"]-sideroads_df.loc[i-1,"length"]
        
sideroads_df.head()        

,road,chainage,lrp,lat,lon,gap,type,name,length
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,0.0
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert,814.0
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101,8.0
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing,178.0
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing,1000.0


In [52]:
bridges_relevant = bridges[["road", "LRPName", "condition", "length", "chainage", "lat", "lon", 'name', 'km','constructionYear']]
bridges_relevant = bridges_relevant.loc[bridges['road'].isin(side_roads)]
bridges_relevant = bridges_relevant.reset_index(drop = True)

bridgestemp = bridges_relevant 

for i in range(1,len(bridges_relevant)):
    if len(str(bridges_relevant["name"][i])) > 4:
        if bridges_relevant["name"][i][-2:] == 'L)' or bridges_relevant["name"][i][-4:] == 'eft)' or bridges_relevant["name"][i][-3:] == 'L )' or bridges_relevant["name"][i][-4:] == 'EFT)':
            bridgestemp = bridgestemp.drop(i,axis = 0)

# Delete depulicates by removing older information
#  assumption: no 2 lrps in the same road have the exact same km
bridges_relevant = bridgestemp \
    .sort_values(by=['road','km','constructionYear'], ascending=False) \
    .drop_duplicates(subset=['road', 'km'], keep='first')


In [53]:
bridges_relevant = bridges_relevant.rename(columns={"LRPName": "lrp"})

# Merge
merged = pd.merge(sideroads_df, bridges_relevant, how="outer", on=["road", "lrp"])
merged = merged.reset_index(drop = True)

# Add model_type
for index, row in merged.iterrows():
    if row["lrp"] == "LRPS" or row["lrp"] == "LRPE":
        merged.loc[index, "model_type"] = "sourcesink"
    else:
        merged.loc[index, "model_type"] = "link"

merged.loc[merged["condition"].notnull(), "model_type"] = "bridge"
merged.head()

,road,chainage_x,lrp,lat_x,lon_x,gap,type,name_x,length_x,condition,length_y,chainage_y,lat_y,lon_y,name_y,km,constructionYear,model_type
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sourcesink
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert,814.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,link
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,link
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing,178.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,link
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,link


In [54]:
merged["chainage_x"] = merged["chainage_x"].fillna(value=merged["chainage_y"])
merged["lat_x"] = merged["lat_x"].fillna(value=merged["lat_y"])
merged["lon_x"] = merged["lon_x"].fillna(value=merged["lon_y"])
merged["name_y"] = merged["name_y"].fillna(value=merged["name_x"])
merged["length_x"] = merged["length_x"].fillna(value=merged["length_y"])

# Keep and rename useful columns only
merged = merged.sort_values(by=['road','chainage_x'],ascending = True)
col_tokeep = ["road", "model_type", "lrp", "name_y", "lat_x", "lon_x", "length_x", "condition","type"]
merged = merged[col_tokeep]
merged = merged.rename(columns={"name_y": "name","lat_x" : "lat", "lon_x" : "lon", "length_x" : "length"})
merged = merged.reset_index(drop = True)

# Add ids
merged["id"] =  range(1000, len(merged) +1000 )
merged.head()

,road,model_type,lrp,name,lat,lon,length,condition,type,id
0,N1,sourcesink,LRPS,Start of Road after Jatrabari Flyover infront...,23.706028,90.443333,0.0,NaN,Others,1000
1,N1,link,LRPSa,Box Culvert,23.702917,90.450417,814.0,NaN,Culvert,1001
2,N1,link,LRPSb,Intersection with Z1101,23.702778,90.450472,8.0,NaN,CrossRoad,1002
3,N1,link,LRP001,Km post missing,23.702139,90.451972,178.0,NaN,KmPost,1003
4,N1,bridge,LRP001a,.,23.698739,90.458861,11.3,A,NaN,1004


In [55]:
cross_sideroads = merged.loc[(merged['type'].str.contains("CrossRoad"))| (merged['type'].str.contains("SideRoad"))]
cross_sideroads = cross_sideroads.reset_index()
cross_sideroads.iloc[60:70,:]

,index,road,model_type,lrp,name,lat,lon,length,condition,type,id
60,936,N1,link,LRP337a,Road to Badarkhali (R172),21.759639,92.076499,575.0,NaN,"SideRoad,Right",1936
61,950,N1,link,LRP341b,Road to Alikadam(Z1005),21.721555,92.083083,163.0,NaN,"SideRoad,Left",1950
62,965,N1,bridge,LRP348a,FULCHARI CULVERT,21.668833,92.075944,96.0,A,"SideRoad,Left",1965
63,985,N1,link,LRP354c,"Z1131, Kutakhali-Maheshkhali",21.615056,92.066777,59.0,NaN,"SideRoad,Right",1985
64,1056,N1,bridge,LRP376a,MUKTAR KUL,21.449472,92.099527,57.0,A,"SideRoad,Left",2056
65,1070,N1,link,LRP378c,Ramu old Section (N109),21.428499,92.095194,79.0,NaN,"SideRoad,Left",2070
66,1112,N1,bridge,LRP386c,DALA-2 SLAB CULVERT,21.425860,92.023166,65.0,A,"SideRoad,Right",2112
67,1213,N1,link,LRP403c,Road to Ramu(Z1009),21.317666,92.093027,82.0,NaN,"SideRoad,Left",2213
68,1217,N1,bridge,LRP404a,UKLIA-SHAHABMARKAT,21.310167,92.097500,220.0,C,"SideRoad,Left",2217
69,1255,N1,bridge,LRP413a,BALUKHALI,21.244750,92.138944,300.0,B,"SideRoad,Left",2255


In [80]:
# show it worked
merged.iloc[690:700,:]

,road,model_type,lrp,name,lat,lon,length,condition,type,id
690,N1,link,LRP259b,Road to Kaliganj (Z1070),22.309694,91.915750,424.0,NaN,CrossRoad,1690
691,N1,link,LRP260,Box culvert,22.311417,91.917638,273.0,NaN,KmPost,1691
692,N1,bridge,LRP260a,PALYA BUS STATION,22.312472,91.918833,168.0,A,Culvert,1692
693,N1,link,LRP260b,Box culvert,22.315639,91.922499,521.0,NaN,Culvert,1693
694,N1,intersection,LRP260c,Road to Kalurghat (N107),22.374638,91.906278,121.0,NaN,"SideRoad,Left",7716
695,N1,bridge,LRP252c,UTTOR SONAPUR,22.315747,91.924352,3.0,A,NaN,1695
696,N1,link,LRP261,Km post missing,22.315556,91.925388,190.0,NaN,KmPost,1696
697,N1,bridge,LRP261a,SRIMI BRIDGE,22.313778,91.932222,750.0,C,Culvert,1697
698,N1,link,LRP262,Km post missing,22.313750,91.934527,250.0,NaN,KmPost,1698
699,N1,link,LRP263,Km post missing,22.313639,91.943889,1000.0,NaN,KmPost,1699


In [85]:
for i, row in merged.iterrows():
    if row['model_type'] != 'bridge':
        for j in range(i,len(merged)):
            if merged['model_type'][j] != 'bridge':
                if row['id'] != merged['id'][j] and row['road'] != merged['road'][j] and distance.euclidean((row['lat'],row['lon']), (merged['lat'][j],merged['lon'][j])) < 0.001:
                    merged.iloc[j,9] = merged.iloc[i,9]
                    merged.iloc[j,4] = merged.iloc[i,4]
                    merged.iloc[j,5] = merged.iloc[i,5]
                    merged.iloc[j,1] = "intersection"
                    merged.iloc[i,1] = "intersection"
                    print('new intersection made at id',row['id'],'between', row['road'], 'and', merged['road'][j])

new intersection made at id 1157 between N1 and Z1042
new intersection made at id 1158 between N1 and Z1042
new intersection made at id 1209 between N1 and N102
new intersection made at id 1210 between N1 and N102
new intersection made at id 7716 between N1 and Z1065
new intersection made at id 1950 between N1 and Z1005
new intersection made at id 2553 between N1 and Z1098
new intersection made at id 2554 between N1 and Z1098
new intersection made at id 3141 between N2 and R211
new intersection made at id 3141 between N2 and R310
new intersection made at id 3377 between N2 and N204
new intersection made at id 3478 between N2 and R240
new intersection made at id 3495 between N2 and N207
new intersection made at id 3824 between N2 and N208
new intersection made at id 5776 between N2 and R250
new intersection made at id 5776 between N2 and R250
new intersection made at id 4234 between N207 and N208
new intersection made at id 4530 between R151 and Z1031
new intersection made at id 4531 be

In [86]:
#--- Adapting to the newest csv guidelines

# Adapting names
#move bridge names to a new column 
merged["bridge_name"] = merged["name"].loc[merged['model_type'] == "bridge"]
#delete names for everything and replace that of SourceSinks according to convention
i = 1 # useful in a second
for index, row in merged.iterrows():
    if not row['model_type'] == "sourcesink":
        merged["name"][index] = ""
        
    elif row['model_type'] == "sourcesink":
        merged["name"][index] = "SoSi" + str(i)
        merged["condition"][index] = ""
        i += 1

# Put columns in right order
merged = merged[["road", "id", "model_type", "condition", "name", "lat", "lon", "length", "bridge_name"]]
merged.head()

C:\Users\nachi\AppData\Local\Temp\ipykernel_14756\1637531242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["name"][index] = "SoSi" + str(i)
C:\Users\nachi\AppData\Local\Temp\ipykernel_14756\1637531242.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["condition"][index] = ""
C:\Users\nachi\AppData\Local\Temp\ipykernel_14756\1637531242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["name"][index] = "

,road,id,model_type,condition,name,lat,lon,length,bridge_name
0,N1,1000,sourcesink,,SoSi1,23.706028,90.443333,0.0,NaN
1,N1,1001,link,NaN,,23.702917,90.450417,814.0,NaN
2,N1,1002,link,NaN,,23.702778,90.450472,8.0,NaN
3,N1,1003,link,NaN,,23.702139,90.451972,178.0,NaN
4,N1,1004,bridge,A,,23.698739,90.458861,11.3,.


In [87]:
# select only the subroads that have a proper intersection to a N1 or N2 road
road_list = [] # empty list for road names to keep
intersect_ids = merged[(merged['road'] == 'N1') | (merged['road'] == 'N2') & (merged['model_type'] == "intersection")]['id'].tolist()
for road in merged['road'].unique().tolist():
    intersections = merged[(merged['road'] == road) & (merged['model_type'] == "intersection")]['id'].tolist()
    if any(intersect in intersect_ids for intersect in intersections):
        road_list.append(road)
merged = merged[merged['road'].isin(road_list)]

In [89]:
for i,row in merged.loc[(merged['id'] == 1756) | (merged['id'] == 5015)].iterrows():
    merged = merged.drop(i)

merged.head()

,road,id,model_type,condition,name,lat,lon,length,bridge_name
0,N1,1000,sourcesink,,SoSi1,23.706028,90.443333,0.0,NaN
1,N1,1001,link,NaN,,23.702917,90.450417,814.0,NaN
2,N1,1002,link,NaN,,23.702778,90.450472,8.0,NaN
3,N1,1003,link,NaN,,23.702139,90.451972,178.0,NaN
4,N1,1004,bridge,A,,23.698739,90.458861,11.3,.


In [92]:
for i, row in merged.iterrows():
    if row['bridge_name'] == '.':
        merged.loc[i, 'bridge_name'] = 'bridge at id '+ str(row['id'])


In [93]:
# Save to csv 
merged.to_csv("../data/processed/N1_N2_plus_sideroads.csv", index=None)